In [65]:
import pickle
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import json
from generator import (SkillsGenerator,
                       DatasetGenerator,
                       AdvancedSkillsGenerator)
from tqdm.notebook import tqdm
from transformers import AutoModel, AutoTokenizer
import gen_prompt_template
from importlib import reload
import pickle
from api_key import API_KEY
import seaborn as sns
tqdm.pandas()


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
## Loading the embedded taxonomy
ESCO_DIR = "../../../../esco/"
with open(ESCO_DIR + "embedded_tech_management_tax.pkl", "rb") as f:
    tech_emb_tax = pickle.load(f)
    tech_emb_tax["embeddings"] = tech_emb_tax["embeddings"].apply(lambda x : x.detach())

## ESCO technical skills
tech_skills = pd.read_csv(ESCO_DIR + "tech_managment_taxonomy_narrow.csv")
tech_skills["name+definition"] = tech_skills["name+defintion"]

## popularity measures
F = pd.read_csv("../ppl_3_sentences_all_skills.csv").drop("Unnamed: 0", axis=1)[["name", "mean_std_ppl"]].set_index("name").to_dict()["mean_std_ppl"]

## Loading the combination dist
MAX_NB_SKILL = 10
combination_dist = np.ones(MAX_NB_SKILL) / MAX_NB_SKILL

## Models (useless for now)
word_emb = "jjzha/jobbert-base-cased"
word_emb_model = AutoModel.from_pretrained(word_emb)
word_emb_tokenizer = AutoTokenizer.from_pretrained(word_emb)
additional_info = tech_skills[["name", "Definition", "Type Level 1", "Type Level 2", "altLabels", "Type Level 3"]].drop_duplicates("name").set_index("name").to_dict("index")


Some weights of BertModel were not initialized from the model checkpoint at jjzha/jobbert-base-cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [67]:
## add category to emb tax
tech_emb_tax_w_cat = tech_emb_tax.merge(tech_skills[["unique_id", "Type Level 1"]], on="unique_id")


gen = AdvancedSkillsGenerator(taxonomy=tech_emb_tax_w_cat, 
                taxonomy_is_embedded=True,
                combination_dist=combination_dist,
                popularity=F,
                entities_category_key="Type Level 1")

datagen = DatasetGenerator(tech_emb_tax,
                           None, ## no references, we work in Zero-Shot
                           word_emb_model,
                           word_emb_tokenizer,
                           additional_info)


In [87]:
args = {
    'nb_generation':10,
    "threshold":0,
    "beam_size":5,
    "temperature_pairing":1,
    "temperature_sample_size":1,
    "frequency_select": True,
    "upper_bound_skill_matching": None
}

list(gen.balanced_nbred_iter(**args))

['Haskell', 'develop energy saving concepts', 'conduct research on flora', 'Incremental development', 'develop terminology databases', 'KDevelop', 'assess railway operations', 'perform a feasibility study for building management systems', 'Absorb (learning management systems)', 'analyse the artistic concept based on stage actions', 'migrate existing data', 'define database physical structure', 'Maltego', 'search historical sources in archives', 'collect property financial information', 'provide context to news stories', 'circuit diagrams', 'sand blasting machine parts', 'assess visual impact of displays', 'check information on prescriptions', 'Erlang', 'taste wines', 'manage ICT data classification', 'manage claim files', 'information structure', 'communicate gambling rules', 'manage temporary ICT networks for live performance', 'check quality of enamel', 'develop policies for nutritional programs', 'SAS language', 'use an application-specific interface', 'audit contractors', 'define f

[['software components suppliers',
  'determine production capacity',
  'ICT system programming',
  'create technical plans',
  'handle case evidence',
  'procurement of ICT network equipment'],
 ['evaluate humanitarian programme proposals',
  'communicate on minerals issues'],
 ['analyse the context of an organisation',
  'analyse internal factors of companies',
  'perform business analysis',
  'circulate information',
  'optimise choice of ICT solution',
  'perform online data analysis',
  'forecast organisational risks',
  'perform PESTEL analysis',
  'identify potential markets for companies'],
 ['set up documentation control system',
  'use computerised maintenance management systems',
  'assist in the development of standard operating procedures in the food chain'],
 ['study court hearings',
  'complete evaluation forms of calls',
  'query languages',
  'provide animal related information for legal proceedings',
  'condense information',
  'digital content creation',
  'present l